# CES DS : Stockage à l'échelle du Web
Stockage Persistance en mémoire / plans d'exécution

Mars 2020

# Préparation
Commencer par **effacer** la sortie de toutes les cellules : Menu Modifier -> Supprimer tous les éléments de sortie


In [1]:
!echo $JAVA_HOME
! java -version


openjdk version "11.0.6" 2020-01-14
OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)


In [2]:
import os

# choisir un mode d'execution: colaboratory, local ou ppti

# Sur colaboratory:
EXECUTION='colaboratory'

# Mode local:
# EXECUTION='local'

# Exécution en salle de TP :
#EXECUTION='ppti'

print('EXECUTION:', EXECUTION)

SPARK_VERSION="2.4.5"

# URL du dossier PUBLIC_DATASET contenant des fichiers des TP
# -----------------------------------------------------------
# en cas de problème avec le téléchargement, aller sur https://nuage.lip6.fr/s/H3bpyRGgnCq2RN4 (Ctrl + click pour suivre le lien)
PUBLIC_DATASET="https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path="

# Ajuster les variables d'environnement selon le mode d'execution
# ----------------------------------------------------------------
if EXECUTION == 'colaboratory':
  HOME="/content"
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  SPARK_PREFIX="/content"
  os.environ["SPARK_HOME"] = f"{SPARK_PREFIX}/spark-{SPARK_VERSION}-bin-hadoop2.7"
  !mkdir -p {os.environ["SPARK_HOME"]}
  DATASET_DIR="/content"

elif EXECUTION == 'local':
  HOME=os.environ["HOME"]
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  SPARK_PREFIX="/local/" + os.environ["USER"]   # ajust this depending on your local 
  os.environ["SPARK_HOME"] = f"{SPARK_PREFIX}/spark-{SPARK_VERSION}-bin-hadoop2.7"  
  # os.environ["SPARK_HOME"] = f"{HOME}/anaconda3/bin/spark-{SPARK_VERSION}-bin-hadoop2.7"  # si pyspark installé par "conda"
  DATASET_DIR=f"{HOME}/dataset"

elif EXECUTION == 'ppti':
  HOME=os.environ["HOME"]
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  SPARK_PREFIX="/usr/local"
  os.environ["SPARK_HOME"] = f"{SPARK_PREFIX}/spark-{SPARK_VERSION}-bin-hadoop2.7"  
  DATASET_DIR="/Infos/bd/spark/dataset"
else:
    print("Valeur incorrecte pour la variable EXECUTION:", EXECUTION)


def checkEnvironmentDirectory(env_variable) :
  if (os.path.isdir(os.environ[env_variable])) :
    print(env_variable, "is correct:", os.environ[env_variable])
  else :
    print(env_variable, "incorrect:", os.environ[env_variable], " directory does not exist! Please fix it")

# check env variabbles are correctly set
checkEnvironmentDirectory("HOME")
checkEnvironmentDirectory("JAVA_HOME")
checkEnvironmentDirectory("SPARK_HOME")

# Check DATASET_DIR
if (os.path.isdir(DATASET_DIR)) :
  print("DATASET_DIR is correct:", DATASET_DIR)
else :
  print("DATASET_DIR incorrect:", DATASET_DIR, "does not exist! Please fix it")

EXECUTION: colaboratory
HOME is correct: /root
JAVA_HOME is correct: /usr/lib/jvm/java-8-openjdk-amd64
SPARK_HOME is correct: /content/spark-2.4.5-bin-hadoop2.7
DATASET_DIR is correct: /content


## Téléchargements

### Spark

In [3]:
import os

if EXECUTION == 'ppti':
  print("on ne telecharge pas spark")  
else:
    if (os.path.isdir(os.environ["SPARK_HOME"] + "/bin")):
        print(os.environ["SPARK_HOME"] + "/bin" + " exists.")
    else:
        CLOSER_LOCATION = f"https://www.apache.org/dyn/closer.lua/spark/spark-{SPARK_VERSION}/spark-{SPARK_VERSION}-bin-hadoop2.7.tgz"
        !wget {CLOSER_LOCATION} -O suggest.html
        !wget `grep "suggest the following mirror" -A2 suggest.html |grep -Po 'href="\K[^"]*'` -O {SPARK_PREFIX}/spark-{SPARK_VERSION}-bin-hadoop2.7.tgz
        !cd {SPARK_PREFIX} && tar xzf spark-{SPARK_VERSION}-bin-hadoop2.7.tgz && rm spark-{SPARK_VERSION}-bin-hadoop2.7.tgz
        !rm suggest.html
        if (os.path.isdir(os.environ["SPARK_HOME"] + "/bin")):
          print(os.environ["SPARK_HOME"] + "/bin" + " ok")
        else:
          print("installation à revoir")

# Rmq: d'autres possibilités pour télécharger spark sont :  !conda install pyspark  ou !pip install pyspark
# La durée de téléchargement (environ 1 minute) est la même quelle que soit la méthode utilisée.

/content/spark-2.4.5-bin-hadoop2.7/bin exists.


### findspark

In [4]:
if EXECUTION == 'ppti' :
  !ls -l /Infos/bd/spark/python/findspark.py
else :
  !pip install findspark

### demarrer_spark.py

In [5]:
if EXECUTION == 'ppti' :
  if os.path.isfile("/Infos/bd/spark/python/demarrer_spark.py") :
    print("demarrer_spark.py exists.")
  else:
    print("ERROR: demarrer_spark.py missing")
elif os.path.isfile("demarrer_spark.py") :
  print("demarrer_spark.py exists.")
else:
  !wget {PUBLIC_DATASET}/python/demarrer_spark.py -O demarrer_spark.py

demarrer_spark.py exists.


### mesure.py

In [6]:
if EXECUTION == 'ppti' :
  if os.path.isfile("/Infos/bd/spark/python/mesure.py") :
    print("mesure.py exists.")
  else:
    print("ERROR: mesure.py missing")
elif os.path.isfile("mesure.py") :
  print("mesure.py exists.")
else:
  !wget {PUBLIC_DATASET}/python/mesure.py -O mesure.py

mesure.py exists.


## Démarrer la session spark

In [7]:
from demarrer_spark import demarrer_spark
from demarrer_spark import chronoRequete
from demarrer_spark import showPartitions
from demarrer_spark import showPartitionSize
from mesure import Mesure

spark = demarrer_spark()

m = Mesure(spark)
# m.mesure(notes1M.distinct())
print("m est défini")

les dossiers contenant les packages

/env/python
/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload
/usr/local/lib/python3.6/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.6/dist-packages/IPython/extensions
/root/.ipython
findspark.init() initialise les variables d'environnement pour spark
import fait
fonction chronoRequete définie
Mesure definie
session démarrée, son id est  local-1588482721907
m est défini


# Mini Projet


## Tâche 1:  Poids des termes dans un document


In [8]:
  # DOSSIER contenant des fichiers
URL="https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path="
# télécharger des fichiers:
dossier = "arxiv"
!mkdir -p {dossier}
fichier = "arxiv_fin_avril_2017.json"
!wget {URL}/{dossier}/{fichier} -O {dossier}/{fichier}

--2020-05-03 05:12:04--  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=/arxiv/arxiv_fin_avril_2017.json
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3488473 (3.3M) [text/plain]
Saving to: ‘arxiv/arxiv_fin_avril_2017.json’

arxiv/arxiv_fin_avr 100%[===================>]   3.33M  4.95MB/s    in 0.7s    

2020-05-03 05:12:06 (4.95 MB/s) - ‘arxiv/arxiv_fin_avril_2017.json’ saved [3488473/3488473]



In [0]:
# DATASET_DIR="/content"
arxiv = spark.read.json(path = DATASET_DIR+"/arxiv/arxiv_fin_avril_2017.json")

In [10]:
arxiv.count()

2650

In [0]:
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *

# # Pour travailler sur un échantillon de données et le cacher en mémoire
# arxiv = arxiv.sample(0.001).persist()
# arxiv.count()

In [12]:
# To be able to use spark.sql syntax
arxiv.createOrReplaceTempView("arxiv")

print("Nombre de documents :", str(arxiv.count()))
arxiv.show(5)

Nombre de documents : 2650
+--------------------+---------+--------------------+--------------------+----------+----+----------+--------------------+
|            abstract|acm-class|             authors|          categories|   created| doi|        id|               title|
+--------------------+---------+--------------------+--------------------+----------+----+----------+--------------------+
|  The task of nex...|     null|[[[, Zhiqian, Zha...|               cs.IR|2017-04-15|null|1704.04576|NEXT: A Neural Ne...|
|  Jet energy loss...|     null|[[[, Shuzhe, Shi,...|hep-ph nucl-ex nu...|2017-04-15|null|1704.04577|A Unified Descrip...|
|  This work consi...|     null|[[[, Jinlong, Lei...|             math.OC|2017-04-15|null|1704.04578|On Synchronous, A...|
|  Chatbots are on...|     null|[[[, Nicole M., R...|         cs.CY cs.SE|2017-04-15|null|1704.04579|Evaluating Qualit...|
|  We present the ...|     null|[[[, Ryoji, Anzak...|physics.atom-ph p...|2017-04-15|null|1704.04583|Fully gener

Définir les objets Dataset suivants en utilisant le langage python et l’API de Spark :

### Doc(docId, termes) où
docId est un identifiant de document et termes est l’ensemble des termes contenus dans le titre et le résumé d’un document.

In [13]:
Doc=spark.sql("  select id, split(concat(title, abstract), ' ') as termes from arxiv   ")
# Doc=arxiv.withColumn('termes', concat(arxiv.title,lit(' '), arxiv.abstract)).select("id","termes")
Doc.show(5)
Doc.count()
# Doc.show(5,truncate=False)

+----------+--------------------+
|        id|              termes|
+----------+--------------------+
|1704.04576|[NEXT:, A, Neural...|
|1704.04577|[A, Unified, Desc...|
|1704.04578|[On, Synchronous,...|
|1704.04579|[Evaluating, Qual...|
|1704.04583|[Fully, general, ...|
+----------+--------------------+
only showing top 5 rows



2650

In [0]:
# docPersistantesParId= Doc.repartition(8,'id').persist()

In [0]:
# showPartitions(docPersistantesParId)

### DocTerme(docId, terme) 
où terme est un terme du document
Rmq vous pouvez utiliser la fonction explode ou flatMap pour réaliser cette étape.

In [16]:
from pyspark.sql.functions import split, explode

# DocTerme=Doc.withColumn("terme",explode("termes")).select("id","terme")
# DocTerme.show(5)

DocTerme=Doc.withColumn("terme",explode("termes")).select("id","terme")
print(DocTerme.count())
DocTerme.show(5)


365549
+----------+---------+
|        id|    terme|
+----------+---------+
|1704.04576|    NEXT:|
|1704.04576|        A|
|1704.04576|   Neural|
|1704.04576|  Network|
|1704.04576|Framework|
+----------+---------+
only showing top 5 rows



In [17]:
#Supprimer certains caractères
# spark.sql(" SELECT regexp_replace(terme, '\"', '') from DocTerme").show(5)
DocTerme=DocTerme.withColumn("termeMod", regexp_replace("terme", "[\"'();:\{\}]", ""))
# DocTerme=DocTerme.withColumn("termeMod", regexp_replace("terme", "[\"'();:\{\}\.,]", "")) #to run next time

# DocTerme.createOrReplaceTempView("DocTermeS")
print(DocTerme.count())

#Ne garder que les mots ayant plus de 3 lettres
DocTerme=DocTerme.where(length(col("termeMod")) >= 3).select("id","termeMod").withColumnRenamed("termeMod","terme")
print(DocTerme.count())
DocTerme.show(5)

365549
294688
+----------+---------+
|        id|    terme|
+----------+---------+
|1704.04576|     NEXT|
|1704.04576|   Neural|
|1704.04576|  Network|
|1704.04576|Framework|
|1704.04576|      for|
+----------+---------+
only showing top 5 rows



### DocTermeFrequence(docId, terme, tf, ntf)
tf est le nombre d’occurrences du terme dans le document
et ntf est la fréquence normalisée : ntf = tf / nombre de termes dans le document

##### Calculer tf

In [18]:
from pyspark.sql.functions import *

CountByTerm=DocTerme.groupBy("id","terme").agg(count("terme").alias('tf'))#.withColumnRenamed("count(terme)","tf")
# CountByTerm=CountByTerm.selectExpr("id","terme","count(terme) as tf")
CountByTerm.show(10)
CountByTerm.count()

+----------+--------------------+---+
|        id|               terme| tf|
+----------+--------------------+---+
|1704.04576|              Neural|  1|
|1704.04576|             Network|  1|
|1704.04576|      Recommendation|  1|
|1704.04576|                task|  1|
|1704.04576|      recommendation|  2|
|1704.04576|                been|  1|
|1704.04576|          developing|  1|
|1704.04576|                 and|  5|
|1704.04576|               users|  2|
|1704.04576|challenging,
because|  1|
+----------+--------------------+---+
only showing top 10 rows



224620

##### Calculer nbr de termes dans le doc

In [19]:
CountByDoc=DocTerme.groupBy("id").agg(count("terme").alias("tsDoc"))#.withColumnRenamed("count(terme)","tsDoc")
nbDocuments=CountByDoc.count()
print("number of documents in the dataset is ",nbDocuments)
CountByDoc.show(10)

number of documents in the dataset is  2650
+----------+-----+
|        id|tsDoc|
+----------+-----+
|1704.04579|  141|
|1704.04590|  145|
|1704.04596|   72|
|1704.04597|   76|
|1704.04598|   79|
|1704.04603|   44|
|1704.04612|  178|
|1704.04614|  138|
|1704.04629|   86|
|1704.04638|  112|
+----------+-----+
only showing top 10 rows



##### join

In [20]:
#Simple Join
DocTermeFrequence=CountByTerm.join(CountByDoc, "id")
print(m.mesure(DocTermeFrequence))
DocTermeFrequence.show(5)

job 15 : shuffle = 3766260 octets, time = 681.0 ms
None
+----------+----------------+---+-----+
|        id|           terme| tf|tsDoc|
+----------+----------------+---+-----+
|1704.04579|        Chatbots|  2|  141|
|1704.04579|             one|  1|  141|
|1704.04579|  conversational|  2|  141|
|1704.04579|agents
activated|  1|  141|
|1704.04579|         execute|  1|  141|
+----------+----------------+---+-----+
only showing top 5 rows



In [0]:
# # broadcast if partitions
# DocTermeFrequence=CountByTerm.join(broadcast(CountByDoc), "id")
# print(m.mesure(DocTermeFrequence))

##### Calculer ntf:tf/nbTermesDoc

In [22]:
DocTermeFrequence=DocTermeFrequence.withColumn("ntf",DocTermeFrequence.tf/DocTermeFrequence.tsDoc).select("id","terme","tf","ntf")
DocTermeFrequence.show(5,truncate=False)
print(m.mesure(DocTermeFrequence))

+----------+----------------+---+---------------------+
|id        |terme           |tf |ntf                  |
+----------+----------------+---+---------------------+
|1704.04579|Chatbots        |2  |0.014184397163120567 |
|1704.04579|one             |1  |0.0070921985815602835|
|1704.04579|conversational  |2  |0.014184397163120567 |
|1704.04579|agents
activated|1  |0.0070921985815602835|
|1704.04579|execute         |1  |0.0070921985815602835|
+----------+----------------+---+---------------------+
only showing top 5 rows

job 18 : shuffle = 3766260 octets, time = 594.0 ms
None


### TermeDF(terme, df, idf) où
df est le nombre de documents contenant le terme
et  idf = log10(D / df) avec D étant le nombre total de documents. Utiliser la fonction log10() après avoir importé les fonctions de pyspark.sql.functions

In [0]:
nbDocTotal=DocTerme.groupBy("id").agg(count("id")).count()

In [24]:
TermeDF=DocTerme.groupBy("terme").agg(count("id").alias("df")) #df : # de document contenant le terme. 
TermeDF=TermeDF.withColumn("idf",log10(nbDocTotal/TermeDF.df)) #nbDocTotal: # total de document
TermeDF.show(5)

+-----------+----+--------------------+
|      terme|  df|                 idf|
+-----------+----+--------------------+
|       NEXT|   5|   2.724275869600789|
|        for|4341|-0.21434391190189206|
|        The|2083|   0.104556603989062|
|        has| 639|  0.6177450157784077|
|extensively|  12|  2.3440646278891832|
+-----------+----+--------------------+
only showing top 5 rows



### DocTermeTFIDF(docId, terme, tfidf) où
tfidf = tf * idf avec  tf associé au couple (docId, terme) et idf est associé au terme.
 


In [25]:
DocTermeTFIDF=DocTermeFrequence.join(TermeDF,'terme').select("id","terme","tf","idf")
# DocTermeTFIDF=DocTermeFrequence.join(broadcast(TermeDF),'terme').select("id","terme","tf","idf")
DocTermeTFIDF.show(5)

+----------+------------+---+------------------+
|        id|       terme| tf|               idf|
+----------+------------+---+------------------+
|1704.07731|         
C2|  1| 3.423245873936808|
|1704.08390|#HashtagWars|  1| 3.423245873936808|
|1704.06943|        $
is|  1| 3.423245873936808|
|1704.07534|      $
such|  1| 3.423245873936808|
|1704.07997|         $$
|  2|3.1222158782728267|
+----------+------------+---+------------------+
only showing top 5 rows



In [26]:
DocTermeTFIDF=DocTermeTFIDF.withColumn("tfidf",DocTermeTFIDF.tf*DocTermeTFIDF.idf).select("id","terme","tfidf").orderBy("tfidf",ascending=False)
DocTermeTFIDF.show(5)

+----------+-----+------------------+
|        id|terme|             tfidf|
+----------+-----+------------------+
|1704.07606| wind|25.970618098177383|
|1704.06853|sleep|25.784710906781015|
|1704.05231|Gabor|24.516415586702422|
|1704.05231|  2-D|24.410384804089713|
|1704.06096|doors|24.232458739368077|
+----------+-----+------------------+
only showing top 5 rows



### DocTermePoidVect(docId, V) où
V est l’ensemble des couples (terme, tfidf) du document



In [27]:
# DocTermePoidVect(docId, V) où
# DocTermePoidVect(docId, V) où
# DocTermeTFIDF.withColumn("coupleTermetfidf","("+DocTermeTFIDF.terme+","+str(DocTermeTFIDF.tfidf)+")").show()
DocTermCouple=DocTermeTFIDF.withColumn('coupleTermetfidf', concat(lit('('),DocTermeTFIDF.terme,lit(','), DocTermeTFIDF.tfidf,lit(')')))
DocTermCouple.show(5)

+----------+-----+------------------+--------------------+
|        id|terme|             tfidf|    coupleTermetfidf|
+----------+-----+------------------+--------------------+
|1704.07606| wind|25.970618098177383|(wind,25.97061809...|
|1704.06853|sleep|25.784710906781015|(sleep,25.7847109...|
|1704.05231|Gabor|24.516415586702422|(Gabor,24.5164155...|
|1704.05231|  2-D|24.410384804089713|(2-D,24.410384804...|
|1704.06096|doors|24.232458739368077|(doors,24.2324587...|
+----------+-----+------------------+--------------------+
only showing top 5 rows



In [28]:
DocTermCouple.groupBy("id").agg(count("coupleTermetfidf").alias("nbCouple")).orderBy('nbCouple',ascending=False).show(3)

+----------+--------+
|        id|nbCouple|
+----------+--------+
|1704.06096|     191|
|1704.05661|     182|
|1704.07934|     179|
+----------+--------+
only showing top 3 rows



In [29]:
DocTermePoidVect=DocTermCouple.groupBy('id').agg(collect_set('coupleTermetfidf').alias('V'))
DocTermePoidVect.show(5)

+----------+--------------------+
|        id|                   V|
+----------+--------------------+
|1704.04579|[(to
the,1.617065...|
|1704.04590|[(Paths,2.9461246...|
|1704.04596|[(appears,3.95217...|
|1704.04597|[(to
higher,3.122...|
|1704.04598|[(uniform,1.85504...|
+----------+--------------------+
only showing top 5 rows



## Tâche 3 : Partitionnement de données et calcul de similarité

### Données de Movielens : films et notes

In [30]:
print("PUBLIC_DATASET", PUBLIC_DATASET)

if os.path.isdir(DATASET_DIR + "/movielens"):
  print(DATASET_DIR + "/movielens exists.")
else:
  !mkdir -p {DATASET_DIR}/movielens

  # DOSSIER contenant des fichiers à télécharger

  # telecharger des fichiers de movielens
  dossier = "movielens"
  !mkdir -p {dossier}

  fichier = "films.json"
  !wget {PUBLIC_DATASET}/{dossier}/{fichier} -O {DATASET_DIR}/{dossier}/{fichier}

  fichier = "notesAMJ.csv"  
  !wget {PUBLIC_DATASET}/{dossier}/{fichier} -O {DATASET_DIR}/{dossier}/{fichier}

  fichier = "notes1M.json"  
  !wget {PUBLIC_DATASET}/{dossier}/{fichier} -O {DATASET_DIR}/{dossier}/{fichier}

  !ls -lh {dossier}

  dossier = "movielens/ml-latest-small"
  !mkdir -p {dossier}
  fichier = "ratings.csv"
  !wget {PUBLIC_DATASET}/{dossier}/{fichier} -O {DATASET_DIR}/{dossier}/{fichier}
  !ls -lh {dossier}
  print("dataset movielens téléchargé")

PUBLIC_DATASET https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=
/content/movielens exists.


In [31]:
# from pyspark.sql.types import *

# Les films
# ---------
films = spark.read.json(path = DATASET_DIR+"/movielens/films.json")\
  .selectExpr("nF as movieId", "titre as title", "g as genres")
films.createOrReplaceTempView("films")

films.printSchema()
films.show(3)

# Les Notes
# ---------
schema_notes = "userId INT, movieId LONG, rating FLOAT, year INT, month INT, day INT"
notes = spark.read.option("header","true")\
    .csv(path = DATASET_DIR+"/movielens/notesAMJ.csv", schema=schema_notes)
notes.createOrReplaceTempView("notes")

notes.printSchema()
notes.show(3)
print("Nombre de notes dans notesAMJ :", str(notes.count()))

# une table avec 1,3 million de notes
notes1M = spark.read.json(path = DATASET_DIR+"/movielens/notes1M.json")\
  .selectExpr("nU as userId", "nF as movieId", "note as rating", "annee as year", "date")
notes1M.createOrReplaceTempView("notes1M")
notes1M.show(3)

print("Nombre de notes dans notes1M :", str(notes1M.count()))

root
 |-- movieId: long (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|   8754|Prime of Miss Jea...|             [Drama]|
| 111486|Lesson of the Evi...|          [Thriller]|
|   1033|Fox and the Hound...|[Animation, Child...|
+-------+--------------------+--------------------+
only showing top 3 rows

root
 |-- userId: integer (nullable = true)
 |-- movieId: long (nullable = true)
 |-- rating: float (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)

+------+-------+------+----+-----+---+
|userId|movieId|rating|year|month|day|
+------+-------+------+----+-----+---+
|   175|     48|   3.0|2003|    5| 14|
|   461|   2001|   3.0|2004|    8|  8|
|   547|  89881|   4.

In [32]:
print("Nombre de notes dans notesAMJ :", str(notes.count()))

Nombre de notes dans notesAMJ : 100004


In [33]:
#Simple check
notes.filter((notes['userId']==175) & (notes['movieId']==48)).show(4)
notes1M.filter((notes1M['userId']==175) & (notes1M['movieId']==48)).show(4)

+------+-------+------+----+-----+---+
|userId|movieId|rating|year|month|day|
+------+-------+------+----+-----+---+
|   175|     48|   3.0|2003|    5| 14|
+------+-------+------+----+-----+---+

+------+-------+------+----+----+
|userId|movieId|rating|year|date|
+------+-------+------+----+----+
+------+-------+------+----+----+



A partir du fichiers notesAMJ.csv de Movielens, on veut obtenir la similarité entre toutes les paires d'utilisateurs ayant une similarité non nulle. On considère qu'un utilisateur a vu les films qu'il a noté et que la similarité se base sur les films vus en commun par deux utilisateurs : la similarité entre u1 et u2 est le nombre de films vus par u1 ET u2 divisé par le nombre de films vus par u1 OU u2. Par exemple si u1 a vu f1, f3 et f4 et u2 a vu f3, f4, f5 et f6 alors la similarité est 2/5 = 0,4.

L'objectif de cette tâche est de comparer deux méthodes de partitionnement des données, par film ou par utilisateur, et d'expliquer les avantages et inconvénients de chaque méthode pour calculer la similarité entre utilisateurs.

In [0]:
# FilmsParUser = spark.sql("""SELECT nU as user, collect_list(nF) as films FROM ratings GROUP BY nU""")
# FilmsParUser.repartition(12, "user").persist()
# FilmsParUser.show(10)
# FilmsParUser.explain()
# print(FilmsParUser.rdd.getNumPartitions())

## Cas1


Cas n°1 : Construire le dataset FilmsParUser(user, films) où user est un numéro d'utilisateur et films et la liste des numéros de films notés par cet utilisateur. Partitionner les données par utilisateur. En supposant que chaque partition est stockée sur une machine différente, expliquer quelles données sont transférées lors du calcul de la similarité entre les paires d'utilisateurs afin d'obtenir le dataset Paire(user1, user2, sim). Expliquer si le calcul s'effectue de la même façon avec un dataset plus grand par exemple avec le fichier ratings2M.zip contenant plus de 270 000 utilisateurs).

### FilmsParUser(user, films): 

Construire le dataset FilmsParUser(user, films) où user est un numéro d'utilisateur et films et la liste des numéros de films notés par cet utilisateur.

In [0]:
# userMovies = spark.sql("""
# SELECT userId, collect_list(movieId) as movieList
# FROM Notes
# GROUP BY userId
# """)
# userMovies.show(4)

In [36]:
notes.show(3)

+------+-------+------+----+-----+---+
|userId|movieId|rating|year|month|day|
+------+-------+------+----+-----+---+
|   175|     48|   3.0|2003|    5| 14|
|   461|   2001|   3.0|2004|    8|  8|
|   547|  89881|   4.0|2011|   10|  8|
+------+-------+------+----+-----+---+
only showing top 3 rows



In [37]:
FilmsParUser=notes.groupBy('userId').agg(collect_set('movieId').alias('films'))
FilmsParUser.show(5)
FilmsParUser.count()

+------+--------------------+
|userId|               films|
+------+--------------------+
|    12|[1220, 3885, 2259...|
|    13|[110, 356, 54286,...|
|    14|[2716, 3988, 3175...|
|    18|[640, 81, 9, 52, ...|
|    25|[1358, 494, 1409,...|
+------+--------------------+
only showing top 5 rows



671

### Partitionner les données par utilisateur.

In [0]:
FilmsParUserPersistantesParId= FilmsParUser.repartition(4,'userId').persist()

In [39]:
showPartitions(FilmsParUserPersistantesParId)

cardinalité des partitions: [164, 176, 163, 168]
Partition 0 :
Row(userId=12, films=[1220, 3885, 2259, 1374, 3827, 3298, 3146, 3879, 1295, 3829, 2355, 737, 3148, 3873, 3831, 608, 2959, 3773, 6184, 529, 1639, 3825, 3869, 1387, 1235, 3798, 3871, 1077, 3865, 3844, 3794, 3176, 673, 3809, 3861, 3324, 3863, 538, 3886, 736, 1215, 3799, 2668, 3474, 3801, 3780, 253, 2460, 1032, 3841, 3791, 3770, 2529, 3864, 1230, 3793, 3408, 1028, 1732, 3179, 1197])
Row(userId=13, films=[110, 356, 54286, 1, 590, 531, 277, 3624, 1265, 2761, 3147, 2355, 7502, 2762, 47, 527, 4310, 88125, 4993, 3396, 58559, 296, 3114, 64614, 480, 81834, 4886, 4306, 524, 1918, 2804, 93363, 4321, 2572, 1259, 69757, 6377, 1027, 2908, 3255, 919, 318, 78499, 362, 4878, 587, 7361, 4718, 5989, 914, 2918, 2571, 1961])
Row(userId=14, films=[2716, 3988, 3175, 2724, 2355, 2683, 3751, 2720, 2394, 2038, 594, 3157, 1196, 3114, 1721, 3354, 3623, 2861, 2628, 3986])
Row(userId=18, films=[640, 81, 9, 52, 785, 743, 74, 786, 765, 32, 736, 25, 76, 707,

In [40]:
showPartitionSize(FilmsParUserPersistantesParId)

cardinalité des partitions: [164, 176, 163, 168]


### Fonction de similarité

In [0]:
@udf('float')
def sim_jaccard(l1, l2):
    set1=set(l1)
    set2=set(l2)
    l = len(set1.union(set2))
    if (l == 0): return 0
    return float(len(set1.intersection(set2)))/len(set1.union(set2))

In [42]:
%%time
userMovies1 = FilmsParUserPersistantesParId.selectExpr("userId as userId1", "films as films1")
userMovies2 = FilmsParUserPersistantesParId.selectExpr("userId as userId2", "films as films2")

Paire = userMovies1.crossJoin(userMovies2).where("films1 > films2").\
  withColumn("sim", sim_jaccard( userMovies1.films1, userMovies2.films2 )).\
  where("sim>0").\
  select("userId1","userId2","sim")

Paire.show(5)

+-------+-------+------------+
|userId1|userId2|         sim|
+-------+-------+------------+
|     12|     13|0.0088495575|
|     12|     18| 0.018181818|
|     12|     50| 0.009433962|
|     12|     56| 0.015679443|
|     12|     67| 0.006134969|
+-------+-------+------------+
only showing top 5 rows

CPU times: user 12.2 ms, sys: 2.37 ms, total: 14.6 ms
Wall time: 2.54 s


### Explain plan execution

En supposant que chaque partition est stockée sur une machine différente, expliquer quelles données sont transférées lors du calcul de la similarité entre les paires d'utilisateurs afin d'obtenir le dataset Paire(user1, user2, sim).

In [43]:
Paire.explain()

== Physical Plan ==
*(4) Project [userId1#828, userId2#832, pythonUDF0#903 AS sim#841]
+- BatchEvalPython [sim_jaccard(films1#829, films2#833)], [userId1#828, films1#829, userId2#832, films2#833, pythonUDF0#903]
   +- *(3) Project [userId1#828, films1#829, userId2#832, films2#833]
      +- *(3) Filter (pythonUDF0#902 > 0.0)
         +- BatchEvalPython [sim_jaccard(films1#829, films2#833)], [userId1#828, films1#829, userId2#832, films2#833, pythonUDF0#902]
            +- CartesianProduct (films1#829 > films2#833)
               :- *(1) Project [userId#563 AS userId1#828, films#756 AS films1#829]
               :  +- *(1) Filter isnotnull(films#756)
               :     +- InMemoryTableScan [films#756, userId#563], [isnotnull(films#756)]
               :           +- InMemoryRelation [userId#563, films#756], StorageLevel(disk, memory, 1 replicas)
               :                 +- ObjectHashAggregate(keys=[userId#563], functions=[collect_set(movieId#564L, 0, 0)])
               :       

In [44]:
m.mesure(Paire)

job 46 : shuffle = 944 octets, time = 287.0 ms


#### Explication du plan d'exécution

Remarque générale: InMemoryTableScan signifie qu'on accède aux données stockées en mémoire grâce au persist().
Tout ce qui est avant InMemoryTableScan explique comment les données ont été mises en mémoire.
C'est à partir de cet étape qu'on commence à analyser le plan d'exécution pour la requête en cours.

Cas 1:
Dans ce cas ou la partitionnement des données en mémoire central se fait pas UerId, il n'y aura pas d'exchange lors des cacluls de la sim.
Car dans chaque partition, il y a des userIds distincts,ie ne sont pas partagé entre les partitions.

Voici le plan d'exécution de la première table du crossJoin:

 +- InMemoryTableScan [films#756, userId#563], [isnotnull(films#756)]
on accède directement au dataset dans FilmsParUserPersistantesParId

- *(1) Project [userId#563 AS userId1#828, films#756 AS films1#829]
on projette les champs qui vont être utilisés: userId1 and films1

Voici le plan d'exécution de la deuxième table du crossJoin:
 +- InMemoryTableScan [films#756, userId#563], [isnotnull(films#756)]
on accède directement au dataset dans FilmsParUserPersistantesParId

+- *(2) Project [userId#563 AS userId2#832, films#756 AS films2#833]
on projette les champs qui vont être utilisés: userId2 and films2

+- CartesianProduct (films1#829 > films2#833)
On calcule le produit cartésien des deux tables avec la condition (films1> films2)

+- BatchEvalPython [sim_jaccard(films1#829, films2#833)], [userId1#828, films1#829, userId2#832, films2#833, pythonUDF0#902]
Spark va appeler et calculer la fonction sim_jaccard ==> pythonUDF0#902

+- *(3) Filter (pythonUDF0#902> 0.0)
le filtre sur la sim est appliqué

+- *(3) Project [userId1#828, films1#829, userId2#832, films2#833]
on ne projette que les champs suivants [userId1#828, films1#829, userId2#832, films2#833] sans la sim

+- BatchEvalPython [sim_jaccard(films1#829, films2#833)], [userId1#828, films1#829, userId2#832, films2#833, pythonUDF0#903]
La sim est appelée encore une autre fois ==> pythonUDF0#903 (cette pythonUDF est différente de celle d'avant #902)

*(4) Project [userId1#828, userId2#832, pythonUDF0#903 AS sim#841]
on projette les champs finaux

### File ratings2M

 Expliquer si le calcul s'effectue de la même façon avec un dataset plus grand par exemple avec le fichier ratings2M.zip contenant plus de 270 000 utilisateurs).

In [0]:
# dossier = "movielens"
# fichier = "ratings2M.zip"
# !wget {PUBLIC_DATASET}/{dossier}/{fichier} -O {DATASET_DIR}/{dossier}/{fichier}

In [0]:
# from pyspark import SparkContext

In [0]:
# import zipfile
# import io

# def zip_extract(x):
#     in_memory_data = io.BytesIO(x[1])
#     file_obj = zipfile.ZipFile(in_memory_data, "r")
#     files = [i for i in file_obj.namelist()]
#     return dict(zip(files, [file_obj.open(file).read() for file in files]))


# sc2=spark.sparkContext
# zips = sc2.binaryFiles(path='movielens/ratings2M.zip',minPartitions=None)
# files_data = zips.map(zip_extract).collect()
# type(files_data)

In [0]:
ratings2M = spark.read.option('header', True).option('separator', ',') \
    .csv(path = 'ratings2M.csv', schema = 'nU INT, nF LONG, note FLOAT, timestamp LONG')\
    .selectExpr("nU as userId", "nF as movieId", "note as rating", "timestamp")

In [49]:
ratings2M.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    110|   1.0|1425941529|
|     1|    147|   4.5|1425942435|
|     1|    858|   5.0|1425941523|
|     1|   1221|   5.0|1425941546|
|     1|   1246|   5.0|1425941556|
+------+-------+------+----------+
only showing top 5 rows



In [50]:
FilmsParUser2M=ratings2M.groupBy('userId').agg(collect_set('movieId').alias('films'))
FilmsParUser2M.show(5)

FilmsParUser2M.count()

+------+--------------------+
|userId|               films|
+------+--------------------+
|    12|[356, 2136, 1730,...|
|    13|[1965, 3258, 5804...|
|    14|[168252, 2959, 31...|
|    18|[912, 81, 1193, 1...|
|    25|[161, 356, 153, 5...|
+------+--------------------+
only showing top 5 rows



20808

In [0]:
ratings2MPersistantesParId= FilmsParUser2M.repartition(4,'userId').persist()

In [52]:
%%time
userMovies1 = ratings2MPersistantesParId.selectExpr("userId as userId1", "films as films1")
userMovies2 = ratings2MPersistantesParId.selectExpr("userId as userId2", "films as films2")

Paire2M = userMovies1.crossJoin(userMovies2).where("films1 > films2").\
  withColumn("sim", sim_jaccard( userMovies1.films1, userMovies2.films2 )).\
  where("sim>0").\
  select("userId1","userId2","sim")


Paire2M.show(5)

+-------+-------+-----------+
|userId1|userId2|        sim|
+-------+-------+-----------+
|     12|     25| 0.02930403|
|     12|     46| 0.07643312|
|     12|     56|0.115207374|
|     12|    126| 0.02097902|
|     12|    135|0.003875969|
+-------+-------+-----------+
only showing top 5 rows

CPU times: user 8.16 ms, sys: 318 µs, total: 8.48 ms
Wall time: 5.54 s


In [53]:
Paire2M.explain()

== Physical Plan ==
*(4) Project [userId1#1014, userId2#1018, pythonUDF0#1103 AS sim#1027]
+- BatchEvalPython [sim_jaccard(films1#1015, films2#1019)], [userId1#1014, films1#1015, userId2#1018, films2#1019, pythonUDF0#1103]
   +- *(3) Project [userId1#1014, films1#1015, userId2#1018, films2#1019]
      +- *(3) Filter (pythonUDF0#1102 > 0.0)
         +- BatchEvalPython [sim_jaccard(films1#1015, films2#1019)], [userId1#1014, films1#1015, userId2#1018, films2#1019, pythonUDF0#1102]
            +- CartesianProduct (films1#1015 > films2#1019)
               :- *(1) Project [userId#941 AS userId1#1014, films#970 AS films1#1015]
               :  +- *(1) Filter isnotnull(films#970)
               :     +- InMemoryTableScan [films#970, userId#941], [isnotnull(films#970)]
               :           +- InMemoryRelation [userId#941, films#970], StorageLevel(disk, memory, 1 replicas)
               :                 +- ObjectHashAggregate(keys=[userId#941], functions=[collect_set(movieId#942L, 0, 0

#### Explication du plan

On a exactement le même plan d'exécution qu'avant, car on a la même structure de données:partition.
Dans le cas n°2, on va justement changer la logique du partitionnement, ce qui va changer le plan d'exécution.

## Cas n°2


Cas n°2 :  Construire le dataset UsersParFilm(film, users) où film est un numéro de film et users et la liste des numéros des utilisateurs ayant noté ce film. Partitionner les données par film. En supposant que chaque partition est stockée sur une machine différente, expliquer quelles données sont transférées lors du calcul de la similarité entre les paires d'utilisateurs. Remarque : l'implémentation de ce cas n°2 avec pyspark est optionnelle.sfdfdsqsffsfsdssdsDocTermCouple.groupBy("id").agg(count("coupleTermetfidf").alias("nbCouple")).orderBy('nbCouple',ascending=False).show(3)zeze



     

In [54]:
print("number of movies",ratings2M.groupBy("movieId").agg(count("movieId")).count())
print("number of users",ratings2M.groupBy("userId").agg(count("userId")).count())

number of movies 23281
number of users 20808


### UsersParFilm(film, users)

Construire le dataset UsersParFilm(film, users) où film est un numéro de film et users et la liste des numéros des utilisateurs ayant noté ce film

In [55]:
UsersParFilm2M=ratings2M.groupBy('movieId').agg(collect_set('userId').alias('users'))
UsersParFilm2M.show(5)
UsersParFilm2M.count()

+-------+--------------------+
|movieId|               users|
+-------+--------------------+
|      2|[2207, 3958, 743,...|
|      4|[15826, 2207, 943...|
|      5|[6215, 15289, 306...|
|      8|[16886, 7893, 152...|
|     12|[5380, 15318, 857...|
+-------+--------------------+
only showing top 5 rows



23281

###  Partitionner les données par film

In [0]:
ratings2MPersistantesParMovie= UsersParFilm2M.repartition(4,'movieId').persist()

In [57]:
showPartitions(ratings2MPersistantesParMovie)

cardinalité des partitions: [5864, 5796, 5690, 5931]
Partition 0 :
Row(movieId=2, users=[2207, 3958, 743, 11131, 12851, 7916, 3371, 18660, 18173, 20024, 9180, 6842, 14371, 6742, 12064, 8493, 10213, 10082, 17255, 9982, 7544, 19262, 17411, 6857, 1435, 16106, 2924, 12435, 2337, 8146, 6295, 4444, 7659, 8923, 9279, 1519, 18303, 14601, 9179, 8692, 10543, 20279, 10312, 3039, 14270, 12419, 17841, 10212, 10568, 2452, 8261, 5923, 13583, 20625, 14972, 14485, 5492, 5005, 18443, 8963, 3054, 15649, 16005, 16492, 17856, 6525, 6425, 2367, 8145, 17525, 19145, 18658, 5476, 13136, 16707, 14400, 4689, 856, 10367, 6309, 400, 20624, 11987, 3740, 9062, 16722, 18086, 12177, 3309, 971, 12333, 8631, 6780, 8144, 17168, 13953, 13466, 5706, 7457, 2035, 13266, 9564, 16119, 14268, 4788, 14068, 13581, 4001, 7572, 1663, 15232, 5490, 19059, 18572, 5390, 4903, 8374, 15903, 8761, 4216, 15447, 13596, 15347, 11514, 15472, 13621, 1316, 12447, 11960, 8289, 19258, 17407, 298, 2505, 6076, 13249, 5620, 5976, 3282, 7240, 5389, 1

### Fonction de similarité

Construire à partir de ratings2MPersistantesParMovie un dataset FilmsParUser(user,films)

In [58]:
ratings2MPersistantesParMovie.show(5)

+-------+--------------------+
|movieId|               users|
+-------+--------------------+
|      2|[2207, 3958, 743,...|
|      4|[15826, 2207, 943...|
|      5|[6215, 15289, 306...|
|      8|[16886, 7893, 152...|
|     12|[5380, 15318, 857...|
+-------+--------------------+
only showing top 5 rows



In [59]:
#explode
FilmParUser2=ratings2MPersistantesParMovie.withColumn("user",explode("users")).select("movieId","user")
FilmParUser2.show()
print("FilmParUser2",FilmParUser2.count())
print("ratings2M",ratings2M.count())
FilmParUser2=FilmParUser2.groupBy('user').agg(collect_set('movieId').alias('films'))
FilmParUser2.show(5)

+-------+-----+
|movieId| user|
+-------+-----+
|      2| 2207|
|      2| 3958|
|      2|  743|
|      2|11131|
|      2|12851|
|      2| 7916|
|      2| 3371|
|      2|18660|
|      2|18173|
|      2|20024|
|      2| 9180|
|      2| 6842|
|      2|14371|
|      2| 6742|
|      2|12064|
|      2| 8493|
|      2|10213|
|      2|10082|
|      2|17255|
|      2| 9982|
+-------+-----+
only showing top 20 rows

FilmParUser2 2000000
ratings2M 2000000
+----+--------------------+
|user|               films|
+----+--------------------+
|  12|[356, 2136, 1730,...|
|  13|[1965, 3258, 5804...|
|  14|[168252, 2959, 31...|
|  18|[912, 81, 1193, 1...|
|  25|[161, 356, 153, 5...|
+----+--------------------+
only showing top 5 rows



In [60]:
FilmParUser2=ratings2MPersistantesParMovie.withColumn("user",explode("users")).groupBy('user').agg(collect_set('movieId').alias('films'))
FilmParUser2.show(5)

+----+--------------------+
|user|               films|
+----+--------------------+
|  12|[356, 2136, 1730,...|
|  13|[1965, 3258, 5804...|
|  14|[168252, 2959, 31...|
|  18|[912, 81, 1193, 1...|
|  25|[161, 356, 153, 5...|
+----+--------------------+
only showing top 5 rows



In [61]:
FilmParUser2.explain()

== Physical Plan ==
ObjectHashAggregate(keys=[user#1336], functions=[collect_set(movieId#942L, 0, 0)])
+- Exchange hashpartitioning(user#1336, 4)
   +- ObjectHashAggregate(keys=[user#1336], functions=[partial_collect_set(movieId#942L, 0, 0)])
      +- Generate explode(users#1143), [movieId#942L], false, [user#1336]
         +- InMemoryTableScan [movieId#942L, users#1143]
               +- InMemoryRelation [movieId#942L, users#1143], StorageLevel(disk, memory, 1 replicas)
                     +- ObjectHashAggregate(keys=[movieId#942L], functions=[collect_set(userId#941, 0, 0)])
                        +- Exchange hashpartitioning(movieId#942L, 4)
                           +- ObjectHashAggregate(keys=[movieId#942L], functions=[partial_collect_set(userId#941, 0, 0)])
                              +- *(1) Project [nU#933 AS userId#941, nF#934L AS movieId#942L]
                                 +- *(1) FileScan csv [nU#933,nF#934L] Batched: false, Format: CSV, Location: InMemoryFileIndex[fi

In [62]:
%%time
movieUsers1 = FilmParUser2.selectExpr("user as user1", "films as films1")
movieUsers2 = FilmParUser2.selectExpr("user as user2", "films as films2")

Paire2 = movieUsers1.crossJoin(movieUsers2).where("films1 > films2").\
  withColumn("sim", sim_jaccard( movieUsers1.films1, movieUsers2.films2 )).\
  where("sim>0").\
  select("user1","user2","sim")
  # orderBy("sim",ascending=False)

Paire2.show(5)

+-----+-----+-----------+
|user1|user2|        sim|
+-----+-----+-----------+
|   12|   25| 0.02930403|
|   12|   56|0.115207374|
|   12|  135|0.003875969|
|   12|  150| 0.18660969|
|   12|  161|0.030927835|
+-----+-----+-----------+
only showing top 5 rows

CPU times: user 7.16 ms, sys: 3.01 ms, total: 10.2 ms
Wall time: 2.98 s


In [63]:
Paire2.explain()

== Physical Plan ==
*(4) Project [user1#1396, user2#1400, pythonUDF0#1516 AS sim#1409]
+- BatchEvalPython [sim_jaccard(films1#1397, films2#1401)], [user1#1396, films1#1397, user2#1400, films2#1401, pythonUDF0#1516]
   +- *(3) Project [user1#1396, films1#1397, user2#1400, films2#1401]
      +- *(3) Filter (pythonUDF0#1515 > 0.0)
         +- BatchEvalPython [sim_jaccard(films1#1397, films2#1401)], [user1#1396, films1#1397, user2#1400, films2#1401, pythonUDF0#1515]
            +- CartesianProduct (films1#1397 > films2#1401)
               :- *(1) Project [user#1336 AS user1#1396, films#1344 AS films1#1397]
               :  +- *(1) Filter isnotnull(films#1344)
               :     +- ObjectHashAggregate(keys=[user#1336], functions=[collect_set(movieId#942L, 0, 0)])
               :        +- Exchange hashpartitioning(user#1336, 4)
               :           +- ObjectHashAggregate(keys=[user#1336], functions=[partial_collect_set(movieId#942L, 0, 0)])
               :              +- Genera

En supposant que chaque partition est stockée sur une machine différente, expliquer quelles données sont transférées lors du calcul de la similarité entre les paires d'utilisateurs. Remarque : l'implémentation de ce cas n°2 avec pyspark est optionnelle.

#### EXPLICATION

Cas 2
InMemoryTableScan signifie qu'on accède aux données stockées en mémoire grâce au persist().
Tout ce qui est avant InMemoryTableScan explique comment les données ont été mises en mémoire.
C'est à partir de cet étape qu'on commence à analyser le plan d'exécution pour la requête en cours.

Voici le plan d'exécution de la première table du crossJoin:

+- InMemoryTableScan [movieId#942L, users#1143]
On regarde à partir de InMemoryTableScan :
On accède aux données stockées dans ratings2MPersistantesParMovie qui a été partitionné en 4 selon les movieId==> (movieId,users(list)) 

+- Generate explode(users#1143), [movieId#942L], false, [user#1336]
On applique la fonction explode à ce dataset, ce qui nous permet d'avoir une ligne par moveId et par userId

+- ObjectHashAggregate(keys=[user#1336], functions=[partial_collect_set(movieId#942L, 0, 0)])
On fait un partial collect_set sur les movieId pour chaque user.
Ce calcul se fait en local sur chaque partition d'ou le partial_collect_set. 
On n'aura que les films contenus dans cette partition avec leurs userId correspondant.
Les données transferées seront: (userId, list(films))

+- Exchange hashpartitioning(user#1336, 4):
On échange les données entre les différentes machines avec la clé userId

+- ObjectHashAggregate(keys=[user#1336], functions=[collect_set(movieId#942L, 0, 0)])
on fait une aggrégation globale par userId, en fusionnant la liste des films. Ce calcul se fait toute partition confondue d'où un collect_set et non pas partial_collect_set.

- *(1) Project [user#1336 AS user1#1396, films#1344 AS films1#1397]
on projette les champs qui vont être utilisés: user1 and films1

Voici le plan d'exécution de la 2ème table du crossJoin
 +- ReusedExchange [user#1336, buf#1364], Exchange hashpartitioning(user#1336, 4)
On réutilise l'aggrégation de données faite dans la première table jusqu'à l'échange des données entre les différentes partitions.
Les deux tables sont semblables.

+- ObjectHashAggregate(keys=[user#1336], functions=[collect_set(movieId#942L, 0, 0)])
on fait une aggrégation globale, toute partition confondue d'où un collect_set et non pas partial_collect_set.
==> on aurait pu accèder directement à cet étape?

+- *(2) Project [user#1336 AS user2#1400, films#1344 AS films2#1401]
on projette les champs qui vont être utilisés: user2 and films2

+- CartesianProduct (films1#1397 > films2#1401)
On calcule le produit cartésien des deux tables avec la condition (films1#1397 > films2#1401)

+- BatchEvalPython [sim_jaccard(films1#1397, films2#1401)], [user1#1396, films1#1397, user2#1400, films2#1401, pythonUDF0#1515]
Spark va appeler et calculer la fonction sim_jaccard ==> pythonUDF0#1515

+- *(3) Filter (pythonUDF0#1515 > 0.0)
le filtre sur la sim est appliqué

+- *(3) Project [user1#1396, films1#1397, user2#1400, films2#1401]
on ne projette que les données suivantes [user1#1396, films1#1397, user2#1400, films2#1401] sans la sim

+- BatchEvalPython [sim_jaccard(films1#1397, films2#1401)], [user1#1396, films1#1397, user2#1400, films2#1401, pythonUDF0#1516]
La sim est appelée encore une autre fois ==> pythonUDF0#1516 (cette pythonUDF est différente de celle d'avant #1515)

*(4) Project [user1#1396, user2#1400, pythonUDF0#1516 AS sim#1409]
on projette les champs finaux